In [110]:
import numpy as np
import os
import statsmodels.stats
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import pairwise_distances
import pickle

In [2]:
savedStats='/media/xinyi/dcis2idc/plots/cnnvaeexp0/cluster_reordered/minibatchkmean_ncluster8n_pcs50epoch311_plottingIdx_progBalanced_0_reordered_plots/neighborHist_withRandom_nsamples40000'

In [85]:
ncluster=8
pvalues=np.array([])
fc=np.array([])
sample=np.array([])
for f in os.listdir(savedStats):
    if not 'significanceSeparated' in f:
        continue
    print(f)
    p_curr=np.genfromtxt(os.path.join(savedStats,f),delimiter=' ').flatten()
    res=np.genfromtxt(os.path.join(savedStats,f[:-26]+'res.csv'),delimiter=' ').flatten()
    randMean=np.genfromtxt(os.path.join(savedStats,f[:-26]+'randMean.csv'),delimiter=' ').flatten()
    fname=f[23:-26]
    sample=np.concatenate((sample,np.repeat(fname,p_curr.size)))
    fc=np.concatenate((fc,res/randMean))

neighborHist_withRandomCancer adjacent normal breast tissue_significanceSeparated.csv
neighborHist_withRandomHyperplasia_significanceSeparated.csv
neighborHist_withRandomAtypical hyperplasia_significanceSeparated.csv
neighborHist_withRandomBreast tissue_significanceSeparated.csv
neighborHist_withRandomDuctal carcinoma in situ and breast tissue_significanceSeparated.csv
neighborHist_withRandomDuctal carcinoma in situ with early infiltratio_significanceSeparated.csv
neighborHist_withRandomDuctal carcinoma in situ_significanceSeparated.csv
neighborHist_withRandomInvasive ductal carcinoma (breast tissue)_significanceSeparated.csv
neighborHist_withRandomInvasive ductal carcinoma and breast tissue_significanceSeparated.csv
neighborHist_withRandomInvasive ductal carcinoma_significanceSeparated.csv
neighborHist_withRandomMicropapillary type ductal carcinoma in situ wi_significanceSeparated.csv


In [90]:
logfc=np.log2(fc)

In [94]:
vmax_abs=np.max(np.abs(logfc))
vmax_abs

3.7083247986093806

In [105]:
for f in np.unique(sample):
    fidx=sample==f
    logfc_f=logfc[fidx].reshape(ncluster,ncluster)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(logfc_f,cmap='seismic',vmin=-vmax_abs,vmax=vmax_abs)
    ax.set_xticks(np.arange(ncluster))
    ax.set_xticklabels(np.arange(ncluster))
    ax.set_yticks(np.arange(ncluster))
    ax.set_yticklabels(np.arange(ncluster))
    ax.tick_params(axis='both', which='major', labelsize=18)
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(savedStats,f+'_fc_noSig.jpg'),dpi=300)
    plt.close()

### separate by pathology clf 

In [203]:
clfsavedir='/media/xinyi/dcis2idc/plots/cnnvaeexp0_pathologyClf_neighbor_clusters_exp0_subset_neighborOnly_crossVal_countAreaNorm_DCIS_AH_H//'
predictions=pd.read_csv(clfsavedir+'/predictions.csv')
predictions

,Unnamed: 0,sampleName,true,predicted
0,0,br1003a_1_cytokeratin_555_aSMA_647_hoechst_A1,Hyperplasia,Hyperplasia
1,1,br1003a_1_cytokeratin_555_aSMA_647_hoechst_A2,Hyperplasia,Hyperplasia
2,2,br1003a_1_cytokeratin_555_aSMA_647_hoechst_A4,Hyperplasia,Hyperplasia
3,3,br1003a_1_cytokeratin_555_aSMA_647_hoechst_A5,Hyperplasia,Hyperplasia
4,4,br1003a_1_cytokeratin_555_aSMA_647_hoechst_A6,Hyperplasia,Hyperplasia
...,...,...,...,...
226,226,br301_7_gh2ax_647_cytokeratin_555_hoechst_D5,DCIS with early infiltration,Hyperplasia
227,227,br301_7_gh2ax_647_cytokeratin_555_hoechst_E1,DCIS with early infiltration,DCIS with early infiltration
228,228,br301_7_gh2ax_647_cytokeratin_555_hoechst_E2,DCIS with early infiltration,DCIS with early infiltration
229,229,br301_7_gh2ax_647_cytokeratin_555_hoechst_E5,DCIS with early infiltration,DCIS with early infiltration


In [204]:
datadir='/media/xinyi/dcis2idc/data'
name='exp0'
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name
sampledir=plotsavepath
clustersavedir_alltrain=os.path.join(sampledir,'cluster_alltrain_reordered')

with open(os.path.join(datadir,'processed','train_cnnvae_names'), 'rb') as input:
    allImgNames=pickle.load(input)
    
with open(os.path.join(datadir,'processed','train_cnnvae_coord'), 'rb') as output:
    coordlist=pickle.load(output)
    
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels'), 'rb') as output:
    cellIDlist=pickle.load(output)
    
br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']

progList=np.copy(allImgNames)
for s in np.unique(allImgNames):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList[allImgNames==s]=prog_s
    
ncluster=8

plotepoch=311
clusterplotdir=os.path.join(clustersavedir_alltrain,'plots')
n_pcs=50
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
with open(os.path.join(clustersavedir_alltrain,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
    clusterRes=pickle.load(output)
    
for p in np.unique(progList):
    if p=='Ductal carcinoma in situ':
        progList[progList==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList[progList==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltratio':
        progList[progList==p]='DCIS with early infiltration'
    
    elif p=='Micropapillary type ductal carcinoma in situ wi':
        progList[progList==p]='DCIS with early infiltration'    
#     elif p=='Atypical hyperlasia':
#         progList[progList==p]='Hyperplasia'
        
progInclude=np.array(['DCIS and breast tissue',  'DCIS with early infiltration', 'Hyperplasia','Atypical hyperplasia'])

progIncludeIdx=np.repeat(False,progList.size)
for p in progInclude:
    progIncludeIdx[progList==p]=True
    
coordlist=coordlist[progIncludeIdx]
allImgNames=allImgNames[progIncludeIdx]
clusterRes=clusterRes[progIncludeIdx]
progList=progList[progIncludeIdx]



In [205]:
clustersavedir_valcores=os.path.join(sampledir,'cluster_valcores_reordered')
clustersavedir_valsamples=os.path.join(sampledir,'cluster_valsamples_reordered')

with open(os.path.join(datadir,'processed','train_cnnvae_coord_valcores'), 'rb') as output:
    coordlist_valcores=pickle.load(output)
with open(os.path.join(datadir,'processed','train_cnnvae_coord_valsamples'), 'rb') as output:
    coordlist_valsamples=pickle.load(output)

savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
with open(os.path.join(clustersavedir_valcores,savenamecluster+'_all'), 'rb') as output:
    clusterRes_valcores=pickle.load(output)
with open(os.path.join(clustersavedir_valsamples,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
    clusterRes_valsamples=pickle.load(output)

with open(os.path.join(datadir,'processed','train_cnnvae_names_valcores'), 'rb') as input:
    allImgNames_valcores=pickle.load(input)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valsamples'), 'rb') as input:
    allImgNames_valsamples=pickle.load(input)
    
br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']

progList_valcores=np.copy(allImgNames_valcores)
for s in np.unique(allImgNames_valcores):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList_valcores[allImgNames_valcores==s]=prog_s
    
progList_valsamples=np.copy(allImgNames_valsamples)
for s in np.unique(allImgNames_valsamples):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList_valsamples[allImgNames_valsamples==s]=prog_s
    
for p in np.unique(progList_valcores):
    if p=='Ductal carcinoma in situ':
        progList_valcores[progList_valcores==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList_valcores[progList_valcores==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltratio':
        progList_valcores[progList_valcores==p]='DCIS with early infiltration'
    
    elif p=='Micropapillary type ductal carcinoma in situ wi':
        progList_valcores[progList_valcores==p]='DCIS with early infiltration'    
#     elif p=='Atypical hyperlasia':
#         progList_valcores[progList_valcores==p]='Hyperplasia'
        
for p in np.unique(progList_valsamples):
    if p=='Ductal carcinoma in situ':
        progList_valsamples[progList_valsamples==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ and breast tissue':
        progList_valsamples[progList_valsamples==p]='DCIS and breast tissue'
    elif p=='Ductal carcinoma in situ with early infiltrati':
        progList_valsamples[progList_valsamples==p]='DCIS with early infiltration'
    
    elif p=='Micropapillary type ductal carcinoma in situ w':
        progList_valsamples[progList_valsamples==p]='DCIS with early infiltration'    
#     elif p=='Atypical hyperlasia':
#         progList_valsamples[progList_valsamples==p]='Hyperplasia'
        
progIncludeIdx_valcores=np.repeat(False,progList_valcores.size)
for p in progInclude:
    progIncludeIdx_valcores[progList_valcores==p]=True
    
progIncludeIdx_valsamples=np.repeat(False,progList_valsamples.size)
for p in progInclude:
    progIncludeIdx_valsamples[progList_valsamples==p]=True
    
coordlist_valcores=coordlist_valcores[progIncludeIdx_valcores]
allImgNames_valcores=allImgNames_valcores[progIncludeIdx_valcores]
clusterRes_valcores=clusterRes_valcores[progIncludeIdx_valcores]
progList_valcores=progList_valcores[progIncludeIdx_valcores]

coordlist_valsamples=coordlist_valsamples[progIncludeIdx_valsamples]
allImgNames_valsamples=allImgNames_valsamples[progIncludeIdx_valsamples]
clusterRes_valsamples=clusterRes_valsamples[progIncludeIdx_valsamples]
progList_valsamples=progList_valsamples[progIncludeIdx_valsamples]



In [206]:
allImgNames_all=np.concatenate((allImgNames,np.concatenate((allImgNames_valcores,allImgNames_valsamples))))
coordlist_all=np.concatenate((coordlist,np.concatenate((coordlist_valcores,coordlist_valsamples),axis=0)),axis=0)
clusterRes_all=np.concatenate((clusterRes,np.concatenate((clusterRes_valcores,clusterRes_valsamples))))
progList_all=np.concatenate((progList,np.concatenate((progList_valcores,progList_valsamples))))

In [207]:
def getHistMatrix(labels,ctlist):
    res=np.zeros((np.unique(labels).size,np.unique(ctlist).size))
    for li in range(res.shape[0]):
        l=np.unique(labels)[li]
        nl=np.sum(labels==l)
        ctlist_l=ctlist[labels==l]
        for ci in range(res.shape[1]):
            c=np.unique(ctlist)[ci]
            res[li,ci]=np.sum(ctlist_l==c)
#             res[li,ci]=np.sum(ctlist_l==c)/nl
        res[li]=res[li]/nl
    return res

In [208]:
neighborhoodSize=16*9
if not os.path.exists(os.path.join(clfsavedir,'neighborhood')):
    os.mkdir(os.path.join(clfsavedir,'neighborhood'))
for p_true in np.unique(predictions['true']):
    print('true: '+p_true)
    for p_pred in np.unique(np.unique(predictions['predicted'][predictions['true']==p_true])):
        print('predicted: '+p_pred)
        
#         coord_p=coordlist[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]
#         cluster_p=clusterRes[progList[plottingIdx_i.astype(int)]==progs[p]]
#         imgNames_p=allImgNames[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]

        selfp=np.array([])
        neighborp=np.array([])
        
        for imgN in np.unique(predictions['sampleName'][np.logical_and(predictions['true']==p_true,predictions['predicted']==p_pred)]):
            print('  '+imgN)
            nsamples=np.sum(allImgNames_all==imgN)
            cluster_ps=clusterRes_all[allImgNames_all==imgN]
            neighbors=np.tile(cluster_ps,(nsamples,1))
            selfs=np.tile(cluster_ps.reshape((-1,1)),(1,nsamples))

            dist=pairwise_distances(coordlist_all[allImgNames_all==imgN],n_jobs=-1)
            distIn=np.logical_and(dist<neighborhoodSize,dist>0)
            selfp=np.concatenate((selfp,selfs[distIn]))
            neighborp=np.concatenate((neighborp,neighbors[distIn]))
        res=getHistMatrix(selfp,neighborp)
        np.savetxt(os.path.join(clfsavedir,'neighborhood',p_true+'2'+p_pred+'.csv'),res)

true: Atypical hyperplasia
predicted: Atypical hyperplasia
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_D3
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_D4
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_D7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C1
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C4
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C5
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C6
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D8
  br1003a_3_collagen1_647_hoechst_C4
  br1003a_3_collagen1_647_hoechst_C5
  br1003a_3_collagen1_647_hoechst_C6
  br1003a_3_collagen1_647_hoechst_D3
  br1003a_3_collagen1_647

  br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B6
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B9
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C10
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C2
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C8
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C9
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D1
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D2
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D5
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D6
  br1003a_3_collagen1_647_hoechst_A1
  br1003a_3_collagen1_647_hoechst_A2
  br1003a_3_collagen1_647_hoechst_A3
  br1003a_3_collagen1_647_hoechst_A5
  br1003a_3_collagen1_647_hoechst_A6
  br1003a_3_collagen1_647_hoechst_A7
  br1003a_3_collagen1_647_hoechst_A8
  br1003a_3_collagen1_647_hoechst_A9
  br1003a_3_collagen1_647_hoechst_B1
  br1003a_3_collagen1_647_hoechst_B10
  br

In [212]:
logfc_all=np.array([])
for p_true in np.unique(predictions['true']):
    print('true: '+p_true)
    trueNeighborhood=np.genfromtxt(os.path.join(clfsavedir,'neighborhood',p_true+'2'+p_true+'.csv'),delimiter=' ')
    for p_pred in np.unique(np.unique(predictions['predicted'][predictions['true']==p_true])):
        if p_pred==p_true:
            continue
        print('predicted: '+p_pred)
        predNeighborhood=np.genfromtxt(os.path.join(clfsavedir,'neighborhood',p_true+'2'+p_pred+'.csv'),delimiter=' ')
        logfc=np.log2(predNeighborhood/trueNeighborhood)
        logfc_all=np.concatenate((logfc_all,logfc.flatten()))
        logfc[logfc==np.inf]=2.38
        fig, ax = plt.subplots(figsize=(10, 10))
        im = ax.imshow(logfc,cmap='seismic',vmin=-2.38,vmax=2.38)
        ax.set_xticks(np.arange(ncluster))
        ax.set_xticklabels(np.arange(ncluster))
        ax.set_yticks(np.arange(ncluster))
        ax.set_yticklabels(np.arange(ncluster))
        ax.tick_params(axis='both', which='major', labelsize=18)
        fig.colorbar(im)
        fig.tight_layout()
        plt.savefig(os.path.join(clfsavedir,'neighborhood',p_true+'2'+p_pred+'_logfc.pdf'))
        plt.close()

true: Atypical hyperplasia
predicted: DCIS and breast tissue
predicted: DCIS with early infiltration
predicted: Hyperplasia
true: DCIS and breast tissue
predicted: Atypical hyperplasia
predicted: DCIS with early infiltration
predicted: Hyperplasia
true: DCIS with early infiltration
predicted: Atypical hyperplasia
predicted: DCIS and breast tissue
predicted: Hyperplasia
true: Hyperplasia
predicted: Atypical hyperplasia
predicted: DCIS and breast tissue
predicted: DCIS with early infiltration


In [211]:
np.max(logfc_all)

2.372497552388522

In [183]:
with open(os.path.join(datadir,'processed','train_cnnvae_names'), 'rb') as input:
    allImgNames=pickle.load(input)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valcores'), 'rb') as input:
    allImgNames_valcores=pickle.load(input)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valsamples'), 'rb') as input:
    allImgNames_valsamples=pickle.load(input)
    
allImgNames_all=np.concatenate((allImgNames,np.concatenate((allImgNames_valcores,allImgNames_valsamples))))


np.unique(allImgNames_all).size

212

In [213]:
neighborhoodSize=16*9
if not os.path.exists(os.path.join(clfsavedir,'clusterComp')):
    os.mkdir(os.path.join(clfsavedir,'clusterComp'))
for p_true in np.unique(predictions['true']):
    print('true: '+p_true)
    for p_pred in np.unique(np.unique(predictions['predicted'][predictions['true']==p_true])):
        print('predicted: '+p_pred)
        
#         coord_p=coordlist[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]
#         cluster_p=clusterRes[progList[plottingIdx_i.astype(int)]==progs[p]]
#         imgNames_p=allImgNames[plottingIdx_i.astype(int)][progList[plottingIdx_i.astype(int)]==progs[p]]

        selfp=np.array([])
        
        for imgN in np.unique(predictions['sampleName'][np.logical_and(predictions['true']==p_true,predictions['predicted']==p_pred)]):
            print('  '+imgN)
            nsamples=np.sum(allImgNames_all==imgN)
            cluster_ps=clusterRes_all[allImgNames_all==imgN]
            selfp=np.concatenate((selfp,cluster_ps))
        uniquec,res=np.unique(selfp,return_counts=True)
        print(uniquec)
        np.savetxt(os.path.join(clfsavedir,'clusterComp',p_true+'2'+p_pred+'.csv'),res)

true: Atypical hyperplasia
predicted: Atypical hyperplasia
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_D3
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_D4
  br1003a_1_cytokeratin_555_aSMA_647_hoechst_D7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C1
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C4
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C5
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C6
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D8
  br1003a_3_collagen1_647_hoechst_C4
  br1003a_3_collagen1_647_hoechst_C5
  br1003a_3_collagen1_647_hoechst_C6
  br1003a_3_collagen1_647_hoechst_D3
  br1003a_3_collagen1_647

  br1003a_2_cytokeratin_555_ki67_647_hoechst_A1
  br1003a_2_cytokeratin_555_ki67_647_hoechst_A10
  br1003a_2_cytokeratin_555_ki67_647_hoechst_A3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_A5
  br1003a_2_cytokeratin_555_ki67_647_hoechst_A7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_A8
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B1
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B2
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B3
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B5
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B6
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B7
  br1003a_2_cytokeratin_555_ki67_647_hoechst_B9
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C10
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C2
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C8
  br1003a_2_cytokeratin_555_ki67_647_hoechst_C9
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D1
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D2
  br1003a_2_cytokeratin_555_ki67_647_hoechst_D5
  br1003a_2_cytokeratin_555_ki67_647_h

In [218]:
logfc_all=np.array([])
for p_true in np.unique(predictions['true']):
    print('true: '+p_true)
    trueNeighborhood=np.genfromtxt(os.path.join(clfsavedir,'clusterComp',p_true+'2'+p_true+'.csv'),delimiter=' ')
    trueNeighborhood=trueNeighborhood/np.sum(trueNeighborhood)
    for p_pred in np.unique(np.unique(predictions['predicted'][predictions['true']==p_true])):
        if p_pred==p_true:
            continue
        print('predicted: '+p_pred)
        predNeighborhood=np.genfromtxt(os.path.join(clfsavedir,'clusterComp',p_true+'2'+p_pred+'.csv'),delimiter=' ')
        predNeighborhood=predNeighborhood/np.sum(predNeighborhood)
        logfc=np.log2(predNeighborhood/trueNeighborhood)
        logfc_all=np.concatenate((logfc_all,logfc.flatten()))
        fig, ax = plt.subplots(figsize=(10, 10))
        im = ax.imshow(logfc.reshape(1,8),cmap='seismic',vmin=-1.47,vmax=1.47)
        ax.set_xticks(np.arange(ncluster))
        ax.set_xticklabels(np.arange(ncluster))
#         ax.set_yticks(np.arange(ncluster))
#         ax.set_yticklabels(np.arange(ncluster))
        ax.tick_params(axis='both', which='major', labelsize=18)
        fig.colorbar(im)
        fig.tight_layout()
        plt.savefig(os.path.join(clfsavedir,'clusterComp',p_true+'2'+p_pred+'_logfc.pdf'))
        plt.close()

true: Atypical hyperplasia
predicted: DCIS and breast tissue
predicted: DCIS with early infiltration
predicted: Hyperplasia
true: DCIS and breast tissue
predicted: Atypical hyperplasia
predicted: DCIS with early infiltration
predicted: Hyperplasia
true: DCIS with early infiltration
predicted: Atypical hyperplasia
predicted: DCIS and breast tissue
predicted: Hyperplasia
true: Hyperplasia
predicted: Atypical hyperplasia
predicted: DCIS and breast tissue
predicted: DCIS with early infiltration


In [217]:
np.max(logfc_all)

1.4640335526638943